In [2]:
import numpy as np
#%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import time
from IPython import display
from IPython.display import clear_output

import glob
import imageio
import PIL
import tensorflow_probability as tfp
import string

from tensorflow.keras import Model
from tensorflow.keras.layers import Layer

from ruamel.yaml import YAML

yaml=YAML()
yaml.default_flow_style=False

In [3]:
# Load dataset, split into test and training data
train_ds = tfds.load('tiny_shakespeare', split= 'train', data_dir="data\\",batch_size=-1)
train_ds_numpy = tfds.as_numpy(train_ds)
train_ds = train_ds_numpy["text"][0].lower()
train_ds = str(train_ds)
train_ds = train_ds.replace("\\n", " ")

#print(train_ds)
train_ds = train_ds.translate(str.maketrans('', '', string.punctuation))

train_ds = train_ds.split()
#print(train_ds)

In [4]:
def assign_id(data):
    num = 1
    global id_dict
    id_dict = {}
    global word_dict
    word_dict = {}
    for i in data:
        if i not in word_dict.keys():
            word_dict[i] = num
            id_dict[f"{num}"] = i
            num+=1
    return

assign_id(train_ds)
print(len(word_dict))
print(len(id_dict))

12073
12073


In [5]:
def to_id(word):
    return word_dict[word]

def from_id(id):
    return id_dict[str(id)]

In [6]:
to_id("castle")
#print(word_dict)

5436

In [7]:
from_id(9)

'me'

In [16]:
def create_context(data,index,width):
    if width%2 != 0:
        print("Width muss gerade sein!")
        return
    contexts = {tuple(data[index]) : [None]*(width+1)}
    #print("context is ",contexts)
    width=int((width/2)*-1)
    for i in range(0,len(contexts[tuple(data[index])])):
        #print(f"width = {int(width)}")
        #print(data[index+width])
        #if index+width >= len(contexts[tuple(data[index])]):
            
        contexts[tuple(data[index])][i] = data[index+width]
        width+=1
    
    for i in range(0,len(contexts[tuple(data[index])])-1):
        if contexts[tuple(data[index])][i] == tuple(data[index]):
            #print(contexts[train_ds[index]])
            contexts[tuple(data[index])].pop(i)

    return contexts

def create_dataset(corpus,width=4):
    for i in range(0,len(corpus)-width):
        ctx = create_context(corpus,i,width)
        #print(ctx.items())
        data = []
        for key in ctx.keys():
            for i in ctx[key]:
                data.append((key,i))
            #print(data)

    dataset = tf.data.Dataset.from_tensor_slices(data)
    return dataset

def onehotcorpus():
    onehot_corpus = [None]*len(word_dict)
    for i in range(0,len(word_dict)):
        vector = [0]*len(word_dict)
        vector[i] = 1
        onehot_corpus[i] = vector
    return onehot_corpus

oc = onehotcorpus()
#print(oc)
for i in oc:
    #print(i)
    break
data = create_dataset(oc)

<TensorSliceDataset shapes: (2, 12073), types: tf.int32>
tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(2, 12073), dtype=int32)
tf.Tensor([0 0 0 ... 0 0 0], shape=(12073,), dtype=int32)


In [9]:
class LSTM(Layer):
  def __init__(self,):
    # no biases!
    super(LSTM, self).__init__()
    self.hidden_layer_1 = tf.keras.layers.Dense(units=300,
                                               activation=None
                                               )
    self.hidden_layer_2 = tf.keras.layers.Dense(units=12073,
                                               activation=tf.keras.activations.sigmoid
                                               )
    self.add_weight


  def build(self, input_shape)


  def call(self,input):
    tf.nn.embedding_lookup()

    return outputs

In [23]:
#Build input pipeline
prefetch_size = tf.data.experimental.AUTOTUNE

# batching, shuffle und prefetching
train_ds = data.batch(64).shuffle(buffer_size=64).prefetch(prefetch_size)

In [ ]:
def plot_performance(total_epochs, train_loss, test_loss, accuracies, timing):
    """Helper function to plot the models performance inline during and after training"""
    #clear_output(wait=True) # Clear the previous graph
    fig, ax = plt.subplots(1,2, figsize=(12,6))
    # Estimation for remaining time
    epoch = len(train_loss) - 1
    remaining_time = (timing[1] - timing[0]) * (total_epochs - epoch)
    fig.suptitle(f"Epoch {epoch} / {total_epochs} - Remaining Training Time: {time.strftime('%M:%S', time.gmtime(remaining_time))} min", fontsize=16)

    ax[0].plot(train_loss)
    ax[0].plot(test_loss)
    ax[0].legend(["training", "test"])
    ax[0].set(xlabel="Training Steps", ylabel="Loss")
    ax[1].plot(accuracies)
    ax[1].set(xlabel="Training Steps", ylabel="Accuracy", title=f"max accuracy: {max(accuracies)*100:.2f}%")

    plt.show()

In [ ]:
def train_step(model, sample, context):
    with tf.GradientTape() as tape:
        prediction = model(data)
        loss = loss_function(target, context)
        gradients = tape.gradient(loss, model.trainable_variables)
  
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  
    return loss 



In [ ]:
def train_step(model, data, target, loss_function, optimizer):
  # loss_object and optimizer_object are instances of respective tensorflow classes
  
  with tf.GradientTape() as tape:
    prediction = model(data, training = True)
    loss = loss_function(target, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
  
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  
  return loss 


def test_step(model, test_data, loss_function):
  # test over complete test data

  test_accuracy_aggregator = []
  test_loss_aggregator = []

  for (data, target) in test_data:
    target = data
    prediction = model(data, training = False)
    
    sample_test_loss = loss_function(target, prediction)
    sample_test_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
    sample_test_accuracy = np.mean(sample_test_accuracy)

    test_loss_aggregator.append(sample_test_loss.numpy())
    test_accuracy_aggregator.append(np.mean(sample_test_accuracy))

  test_loss = np.mean(test_loss_aggregator)
  test_accuracy = np.mean(test_accuracy_aggregator)

  return test_loss, test_accuracy

In [ ]:
tf.keras.backend.clear_session()

### Hyperparameters
num_epochs = 10
learning_rate = tf.optimizers.schedules.ExponentialDecay(0.001, 5000, 0.97, staircase=True) #polynomial?
running_average_factor = 0.95

# Initialize the model.
model = Auto_encoder()


# Initialize the loss: categorical cross entropy. Check out 'tf.keras.losses'.
# loss_func = tf.keras.losses.BinaryCrossentropy()
loss_func = tf.keras.losses.MeanSquaredError()

# Initialize the optimizer: Adam with default parameters. Check out 'tf.keras.optimizers'
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Initialize lists for later visualization.
train_losses = []
test_losses = []
test_accuracies = []

#
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(f"Training started at {current_time}.")

#testing once before we begin
test_loss, test_accuracy = test_step(model, test_ds, loss_func)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

#check how model performs on train data once before we begin
train_loss, _ = test_step(model, train_ds, loss_func)
train_losses.append(train_loss)
print(model.summary())

# We train for num_epochs epochs.
for epoch in range(num_epochs):
    start = time.perf_counter()
    print('Epoch: __ ' + str(epoch))

    #training (and checking in with training)
    running_average = 0
    for (data,target) in train_ds:
        #train_loss = train_step(model, data, target, loss_func, optimizer)
        train_loss = train_step(model, data, data, loss_func, optimizer)
        running_average = running_average_factor * running_average  + (1 - running_average_factor) * train_loss
    train_losses.append(running_average)

    #testing
    test_loss, test_accuracy = test_step(model, test_ds, loss_func)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
    
    end = time.perf_counter()
    plot_performance(num_epochs, train_losses, test_losses, test_accuracies, (start, end))

t2 = time.localtime()
current_time2 = time.strftime("%H:%M:%S", t)
t_delta = time.mktime(t2)-time.mktime(t)
print(f"Training ended at {current_time}. Duration was {t_delta/60:.2f} minutes.")


# images = model(data,training=False).numpy()
# tile_plot(images,labelling=False)